In [1]:
from abc import ABC, abstractmethod

In [2]:
class AccountInterface(ABC):
    def __init__(self):
        super().__init__()
    
    @abstractmethod
    def deposit(self, amount: float) -> float:
        raise NotImplementedError
    
    @abstractmethod
    def withdraw(self, amount: float) -> str:
        raise NotImplementedError
    
    @abstractmethod
    def get_balance(self) -> float:
        raise NotImplementedError


In [3]:
class State(ABC):
    def __init__(self):
        super().__init__()
    
    @abstractmethod
    def get_state_name(self) -> str:
        raise NotImplementedError
    
    @abstractmethod
    def withdraw(self, amount: float) -> str:
        raise NotImplementedError
    
    @abstractmethod
    def deposit(self, amount: float) -> str:
        raise NotImplementedError

In [4]:
class ServiceFeesInterface(ABC):
    def __init__(self):
        super().__init__()
    
    @abstractmethod
    def service_charge(self, amount: float) -> float:
        raise NotImplementedError
    

In [5]:
class Account(AccountInterface):
    
    def __init__(self, service_fees: ServiceFeesInterface):
        # Instance Variable
        self._service_fees = service_fees
        self._lock_state = None
        self._normal_state = None
        self._current_state = None
        self._balance = 0
    
    # Override
    def deposit(self, amount: float) -> str:
        print('=== DEPOSIT ===')
        return (self._current_state).deposit(amount)
    
    # Override
    def withdraw(self, amount: float) -> str:
        print('=== WITHDRAW ===')
        return (self._current_state).withdraw(amount)
    
    # Override
    def get_balance(self) -> float:
        return round(self._balance, 2)
    
    def set_state(self, state: State) -> None:
        self._current_state = state
    
    def set_up_normal_state(self, normal_state: State) -> State:
        self._normal_state = normal_state
        
    def set_up_lock_state(self, lock_state: State) -> State:
        self._lock_state = lock_state
        self._current_state = self._lock_state
    
    # @proprety is to access getter, method to call self.normal_state
    @property
    def normal_state(self) -> State:
        return self._normal_state
    
    @property
    def lock_state(self) -> State:
        return self._lock_state
    
    @property
    def current_state(self) -> State:
        return self._current_state
      
    def get_service_charge(self, amount: float) -> float:
        return (self._service_fees).service_charge(amount)
    
    def increase_balance(self, amount: float) -> None:
        self._balance += amount
        
    def decrease_balance(self, amount: float) -> None:
        self._balance -= amount

In [6]:
class NormalState(State):
    
    def __init__(self, account: Account):
        self._account = account
        (self._account).set_up_normal_state(self)
    
    # Override
    def get_state_name(self) -> str:
        return 'Normal State'
    
    # Override
    def withdraw(self, amount: float) -> str:
        charges = (self._account).get_service_charge(amount)
        total = charges + amount
        
        if (amount >= 1000 and total <= (self._account).get_balance()):
            (self._account).decrease_balance(total)
            self.__update_state()
            return 'RM' + str(amount) + ' with additional charges RM' + str(charges) + ' is deduct from the account\nCurrent Account State:' + (self._account).current_state.get_state_name() + '\nAccount Balance: RM' + str((self._account).get_balance()) + '\n';
        elif (amount < 1000 and amount <= (self._account).get_balance()):
            (self._account).decrease_balance(amount)
            self.__update_state()
            return 'RM' + str(amount) +' is deduct from the account\nCurrent Account State: ' + (self._account).current_state.get_state_name() + '\nAccount Balance: RM' + str((self._account).get_balance()) + '\n'
        else:
            return 'Sorry, Withdraw amount is more than account balance.\nWithdraw transaction is denied.'
        
    # Override
    def deposit(self, amount: float) -> str:
        (self._account).increase_balance(amount)
        return 'RM' + str(amount) + ' is added to the account\nCurrent Account State: ' + (self._account).current_state.get_state_name() + '\nAccount Balance: RM' + str((self._account).get_balance()) + '\n';
    
    def __update_state(self) -> None:
        if((self._account).get_balance() < 50):
            (self._account).set_state((self._account).lock_state)

In [7]:
class LockState(State):
    
    def __init__(self, account: Account):
        self._account = account
        (self._account).set_up_lock_state(self)
    
    # Override
    def get_state_name(self) -> str:
        return 'Lock State'
    
    # Override
    def withdraw(self, amount: float) -> str:
        return 'Sorry, Unable to perform withdrawal. Account should have minimum RM50 in order to perform withdrawal.\nCurrent Account State: ' + (self._account).current_state.get_state_name()  + '\nAccount Balance: RM' + str((self._account).get_balance()) + '\n'
        
    # Override
    def deposit(self, amount: float) -> str:
        (self._account).increase_balance(amount)
        self.__update_state()
        return 'RM' + str(amount) + ' is added to the account\nCurrent Account State: ' + ((self._account).current_state).get_state_name() + '\nAccount Balance: RM' + str((self._account).get_balance()) + '\n'
    
    def __update_state(self) -> None:
        if((self._account).get_balance() >= 50):
            (self._account).set_state((self._account).normal_state)

In [8]:
class ServiceFees(ServiceFeesInterface):
    
    def __init__(self):
        pass
    
    # Override
    def service_charge(self, amount: float) -> float:
        return amount* 0.01

In [9]:
def inputValidation(input: str) -> float:
    try :
        return float(input);
    except ValueError :
        return -1;

In [10]:
def optionValidation(input: str) -> int:
    try :
        return int(input);
    except Exception as e :
        return 5;

In [11]:
def main():
    print('Hello World Bank (Single)')

    account = Account(ServiceFees())
    LockState(account)
    NormalState(account)

    print('Welcome to Hello World Bank :)')

    while(True):
        option = optionValidation(input('1. Deposit\n2. Withdraw\n3. View Balance\n4. Exit\nChoose Option (1 - 4):'))

        if(option == 1):
            amount = inputValidation(input("Please enter deposit amount:"))

            while(amount < 1):
                if amount == 0:
                    amount = inputValidation(input('Deposit amount cannot be 0 or less than 0. Please enter the deposit amount: '))
                else:
                    amount = inputValidation(input('Invalid amount. Please enter the deposit amount: '))

            print(account.deposit(amount))

        elif(option == 2):
            amount = inputValidation(input('Please enter withdrawal amount:\n(Note: 1% of addition service charge will be charged if the withdrawal amount more than or equal to RM1000)\n'))
            while(amount < 1):
                if amount == 0:
                    amount = inputValidation(input('Withdraw amount cannot be 0 or less than 0. Please enter the withdraw amount: '))
                else:
                    amount = inputValidation(input('Invalid amount. Please enter the withdraw amount: '))
            print(account.withdraw(amount))
            
        elif(option == 3):
            print('Account Balance: RM', str(account.get_balance()))

        elif(option == 4):
            print('Thank you for choosing Hello World Bank. Have a nice day! Thank you. :)')
            break

        else:
            print('Opps,Invalid option. Please choose again.')

In [12]:
if __name__ == "__main__": 
    main()

Hello World Bank (Single)
Welcome to Hello World Bank :)
1. Deposit
2. Withdraw
3. View Balance
4. Exit
Choose Option (1 - 4):4
Thank you for choosing Hello World Bank. Have a nice day! Thank you. :)
